### titanic 예측
- titanic_cleaning.csv 파일로 진행
- PassengerId 는 순차적인 번호이므로 제외하고 진행
- train, test 80:20 wlsgod
- 여러가지 모델 생성 후 최적의 모델 이용
- 최적의 모델츨 찾고 자신의 인적사항 넣고 결과 예측

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

df = pd.read_csv("data/titanic_cleaning.csv")
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,0,22.0,1,0,7.2500
1,2,1,1,1,38.0,1,0,71.2833
2,3,1,3,1,26.0,0,0,7.9250
3,4,1,1,1,35.0,1,0,53.1000
4,5,0,3,0,35.0,0,0,8.0500


#### feature, lable 분리

In [2]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare'],
      dtype='object')

In [4]:
feature = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch','Fare']
label = 'Survived'
X = df[feature]
y = df[label]

- 훈련, 테스트 셋 분리

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[feature], df[label], test_size=0.2)
print("총 개수 : ", X.shape, y.shape)
print("train 개수 : ", X_train.shape, y_train.shape)
print("test 개수 : ", X_test.shape, y_test.shape)

총 개수 :  (891, 6) (891,)
train 개수 :  (712, 6) (712,)
test 개수 :  (179, 6) (179,)


#### Kneighbors(k-최근접 이웃 알고리즘)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(X_train, y_train)
kn.score(X_test, y_test)

0.6759776536312849

#### svm(Support Vector Machine)
- linear / rbf

In [7]:
import sklearn.svm as svm
svm_linear = svm.SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
svm_linear.score(X_test, y_test)

0.8212290502793296

In [8]:
svm_rbf = svm.SVC(kernel='rbf')
svm_rbf.fit(X_train, y_train)
svm_rbf.score(X_test, y_test)

0.6536312849162011

#### Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_clf.score(X_test, y_test)

0.8100558659217877

#### 보팅(Voting)
- 소프트 / 하드

In [10]:
svm_rbf = svm.SVC(kernel='rbf', probability=True)
svm_rbf.fit(X_train, y_train)

kn = KNeighborsClassifier();
kn.fit(X_train, y_train)

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [11]:
from sklearn.ensemble import VotingClassifier

vo = VotingClassifier(estimators=[("svc", svm_rbf), ("knn", kn), ("DecisionTree", dt_clf)], voting="soft")

vo.fit(X_train, y_train)

print("svm : ", svm_rbf.score(X_test, y_test))
print("kn : ", kn.score(X_test, y_test))
print("dt_clf : ", dt_clf.score(X_test, y_test))
print("vo : ", vo.score(X_test, y_test))

svm :  0.6536312849162011
kn :  0.6759776536312849
dt_clf :  0.8100558659217877
vo :  0.7821229050279329


#### 앙상블 - 배깅(Bagging)

In [12]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.8491620111731844

#### 앙상블 - 부스팅(Boosting)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
gbc.score(X_test, y_test)

0.8603351955307262

### 생존, 죽은 여부 확인
- Survived : 0이면 사망, 1이면 생존
- Pclass : 티켓등급
- Sex : 성별(0:남성, 1:여성)
- Age : 나이
- SibSp : 형제, 자매, 배우자의 합
- Parch : 부모, 자식의 합
- Fare : 요금

In [14]:
test = [[1, 0, 35, 2, 3, 20]]
result = rfc.predict(test)
result

array([0], dtype=int64)

In [15]:
if(result[0]==1):
    print("생존 성공")
else:
    print("죽음")

죽음
